In [ ]:
import torch
import numpy as np

# # check if CUDA is available
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 64
# percentage of training set to use as validation
valid_size = 0.2

#  rotation, translation,
# flipping and zooming
# convert data to a normalized torch.FloatTensor 
train_transforms = transforms.Compose([
#                                        transforms.ColorJitter(hue=.50, saturation=.50),
                                       transforms.RandomRotation(0.3),
                                       transforms.RandomPerspective(0.3),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.Resize(100),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                       transforms.RandomErasing()])

transforms = transforms.Compose([transforms.Resize((100,100)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    
# choose the training and test datasets
train_data = datasets.ImageFolder('../input/my-cats-dogs-dataset-sorted/train/train', transform=train_transforms)
# test_data = TestDataset(data_dir + '/test', transform=transform)

In [ ]:
train_data

In [ ]:
# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

In [ ]:
# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
#     num_workers=num_workers)

# specify the image classes
classes = train_data.classes; train_data.classes

### Visualize a Batch of Training Data

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

In [ ]:
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

In [ ]:
len(train_data.classes)

In [ ]:
images.shape

In [ ]:
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
# display 10 images
for idx in np.arange(20):
    ax = fig.add_subplot(2, int(20/2), idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

In [ ]:
import torch.nn as nn
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 128, 2, padding=1)
        self.norm1 = nn.BatchNorm2d(128)

        self.conv2 = nn.Conv2d(128, 160, 2, padding=1)
        self.norm2 = nn.BatchNorm2d(160)
        
        self.conv3 = nn.Conv2d(160, 256, 2, padding=1)
        self.norm3 = nn.BatchNorm2d(256)
        
        self.conv4 = nn.Conv2d(256, 324, 2, padding=1)
        self.norm4 = nn.BatchNorm2d(324)

        self.conv5 = nn.Conv2d(324, 256, 2, padding=1)
        self.norm5 = nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(256, 128, 2, padding=1)
        self.norm6 = nn.BatchNorm2d(128)

        # max pooling layer
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(1, 1)

        self.fc1 = nn.Linear(128 * 5 * 5, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 1)
        
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        
        x = self.norm1(self.pool1(F.leaky_relu(self.conv1(x))))
        
#         x = self.dropout(x)
        
        x = self.norm2(self.pool1(F.leaky_relu(self.conv2(x))))
        
#         x = self.dropout(x)
        
        x = self.norm3(self.pool1(F.leaky_relu(self.conv3(x))))
        
#         x = self.dropout(x)
        
        x = self.norm4(self.pool1(F.leaky_relu(self.conv4(x))))
        
#         x = self.dropout(x)
        
        x = self.norm5(self.pool1(F.leaky_relu(self.conv5(x))))
        
#         x = self.dropout(x)
        
        x = self.norm6(self.pool2(F.leaky_relu(self.conv6(x))))
        

#         x = self.dropout(x)

#         # flatten image input
        x = x.view(-1, 128 * 5 * 5)

        x = F.leaky_relu(self.fc1(x))

#         x = self.dropout(x)

        x = F.leaky_relu(self.fc2(x))

#         x = self.dropout(x)
        
        x = F.leaky_relu(self.fc3(x))

#         x = self.dropout(x)

        x = torch.sigmoid(self.fc4(x))

        return x

# create a complete CNN
model = Net()
print(model)

In [ ]:
images, labels = dataiter.next()
images, labels = images.to(device), labels.to(device)

In [ ]:
# specify loss function
# criterion = nn.CrossEntropyLoss()

# specify optimizer
# optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

from torch import optim
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-4)
model.to(device)

---
## Train the Network

In [ ]:
accuracy = []
model.eval()
with torch.no_grad():
    for data, target in valid_loader:
        data, target = data.to(device), target.float().view(-1,1).to(device)
        output = model(data)
        preds = (output > 0.5).float()
        accuracy_part = (preds == target).float()
        accuracy.extend(accuracy_part)
accuracy_ = torch.mean(torch.tensor(accuracy))
print(f'Val Accuracy Overall: {accuracy_}')

In [ ]:
model.load_state_dict(torch.load('/kaggle/working/model_cats_dogs.pt'))

In [ ]:
accuracy = []
model.eval()
with torch.no_grad():
    for data, target in valid_loader:
        data, target = data.to(device), target.float().view(-1,1).to(device)
        output = model(data)
        preds = (output > 0.5).float()
        accuracy_part = (preds == target).float()
        accuracy.extend(accuracy_part)
accuracy_ = torch.mean(torch.tensor(accuracy))
print(f'Val Accuracy Overall: {accuracy_}')

In [ ]:
# # number of epochs to train the model
# n_epochs = 80 # you may increase this number to train a final model

# valid_loss_min = 0.036545 # track change in validation loss

# for epoch in range(1, n_epochs+1):

#     # keep track of training and validation loss
#     train_loss = 0.0
#     valid_loss = 0.0
    
#     ###################
#     # train the model #
#     ###################
#     model.train()
#     for data, target in train_loader:
#         data, target = data.to(device), target.float().view(-1,1).to(device)
#         # clear the gradients of all optimized variables
#         optimizer.zero_grad()
#         # forward pass: compute predicted outputs by passing inputs to the model
#         output = model(data)
#         # calculate the batch loss
#         loss = criterion(output, target)
#         # backward pass: compute gradient of the loss with respect to model parameters
#         loss.backward()
#         # perform a single optimization step (parameter update)
#         optimizer.step()
#         # update training loss
#         train_loss += loss.item()*data.size(0)
        
#     ######################    
#     # validate the model #
#     ######################
#     model.eval()
#     with torch.no_grad():
#         for data, target in valid_loader:
#             # move tensors to GPU if CUDA is available
#             data, target = data.to(device), target.float().view(-1,1).to(device)
#             # forward pass: compute predicted outputs by passing inputs to the model
#             output = model(data)
#             # calculate the batch loss
#             loss = criterion(output, target)
#             # update average validation loss 
#             valid_loss += loss.item()*data.size(0)
    
#     # calculate average losses
#     train_loss = train_loss/len(train_loader.dataset)
#     valid_loss = valid_loss/len(valid_loader.dataset)
        
#     # print training/validation statistics 
#     print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
#         epoch, train_loss, valid_loss))
    
#     # save model if validation loss has decreased
#     if valid_loss <= valid_loss_min:
#         print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
#         valid_loss_min,
#         valid_loss))
#         torch.save(model.state_dict(), '/kaggle/working/model_cats_dogs.pt')
#         valid_loss_min = valid_loss

In [ ]:
model.load_state_dict(torch.load('/kaggle/working/model_cats_dogs.pt'))

In [ ]:
import pandas as pd

In [ ]:
submission = pd.read_csv('../input/asdfsdfasdfasf/sampleSubmission.csv'); submission

In [ ]:
# test_data = datasets.ImageFolder('../input/my-cats-dogs-dataset-sorted/test', transform=transforms)

In [ ]:
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, num_workers=num_workers)

In [ ]:
test_loader.dataset

In [ ]:
test_loader.dataset.classes

In [ ]:
# array_of_preds = []
# for data, target in test_loader:
#     data, target = data.cuda(), target.cuda()
#     output = model(data)
#     preds = (output > 0.5).float()
#     array_of_preds.extend(preds.cpu().detach().numpy())

In [ ]:
# submission['label'] = array_of_preds

In [ ]:
submission['label'] = submission['label'].astype('int64')

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
pd.read_csv('/kaggle/working/submission.csv')